Notebook for plotting hillshade or slope rasters on 3D surface plot of elevation with Plotly

based off this tutorial https://plot.ly/~empet/14172/mapping-an-image-on-a-surface/#/

In [17]:
# Import necessary libraries

import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import rasterio as rio
import rasterio.plot
import pandas as pd
import itertools
from plotly_func import raster_for_plotly

In [20]:
# Read in elevation surfaces for plotly

LiDAR_dem = raster_for_plotly(rio.open(LiDAR_dem_p))
LiDAR_UAS_dem = raster_for_plotly(rio.open(LiDAR_UAS_dem_p))
#UAS_dsm = raster_for_plotly(rio.open(UAS_dsm_p))

In [6]:
# Read in hillshade with rasterio and get arrays

LiDAR_dem_hs = rio.open(LiDAR_dem_hs_p)
LiDAR_UAS_dem_hs = rio.open(LiDAR_UAS_dem_hs_p)
#LiDAR_dsm_hs = rio.open(LiDAR_dsm_hs_p)
#UAS_hs = rio.open(UAS_hs_p)

LiDAR_dem_hs_arr = LiDAR_dem_hs.read()[0]
LiDAR_UAS_hs_arr = LiDAR_UAS_dem_hs.read()[0]
#UAS_hs_arr = UAS_hs.read()[0]

In [7]:
surfcolor = LiDAR_dem_hs_arr
surf2color = LiDAR_UAS_hs_arr
#surf3color = UAS_hs_arr

In [8]:
# Function to convert matplotlib color ramp to plotly color ramp

def mpl_to_plotly(cmap, pl_entries):
    h=1.0/(pl_entries-1)
    pl_colorscale=[]
    for k in range(pl_entries):
        C=list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([round(k*h,3), 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale

In [9]:
pl_grey=mpl_to_plotly(cm.Greys_r, 256)

In [10]:
surf=Surface(x=LiDAR_dem['x'], 
             y=LiDAR_dem['y'], 
             z=LiDAR_dem['z'],
             colorscale=pl_grey,
             surfacecolor=surfcolor,
             showscale=False
            )

In [11]:
surf2=Surface(x=LiDAR_UAS_dem['x'], 
             y=LiDAR_UAS_dem['y'], 
             z=LiDAR_UAS_dem['z'],
             colorscale=pl_grey,
             surfacecolor=surf2color,
             showscale=False
            )

In [12]:
# Get Extent

# Find x, y, z, max and min values
x_min = min([min(LiDAR_dem['x']), min(LiDAR_UAS_dem['x'])])
x_max = max([max(LiDAR_dem['x']), max(LiDAR_UAS_dem['x'])])
y_min = min([min(LiDAR_dem['y']), min(LiDAR_UAS_dem['y'])])
y_max = max([max(LiDAR_dem['y']), max(LiDAR_UAS_dem['y'])])
z_min = min([min(LiDAR_dem['z'].flatten()[~np.isnan(LiDAR_dem['z'].flatten())]),min(LiDAR_UAS_dem['z'].flatten()[~np.isnan(LiDAR_UAS_dem['z'].flatten())])])
z_max = max([max(LiDAR_dem['z'].flatten()[~np.isnan(LiDAR_dem['z'].flatten())]),max(LiDAR_UAS_dem['z'].flatten()[~np.isnan(LiDAR_UAS_dem['z'].flatten())])])

# Create points to plot in corners
A = [x_min, x_max]
B = [y_min, y_max]
C = [z_min, z_max]

a = [A, B, C]

# Create DataFrame of unique combination of points

extent_df = pd.DataFrame(list(itertools.product(*a)), columns = ['x', 'y', 'z'])

extent_trace = trace1 = go.Scatter3d(
    x=extent_df['x'],
    y=extent_df['y'],
    z=extent_df['z'],
    opacity = 0
)

In [13]:
# Create Buttons

LiDAR_dem_b = [True, False]
LiDAR_UAS_b = [False, True]
#UAS_dsm_b = [False, False, True]

updatemenus = list([
    dict(type="buttons",
         active=1,
         buttons=list([
            dict(label = 'Pre-Slide LiDAR DEM',
                 method = 'update',
                 args = [{'visible': LiDAR_dem_b},
                         {'title': 'Pre-Slide LiDAR DEM'}]),
             dict(label = 'Post-Slide Blended DEM',
                 method = 'update',
                 args = [{'visible': LiDAR_UAS_b},
                         {'title': 'Post-Slide Blended DEM'}])
#              ,
#              dict(label = 'UAS DSM',
#                  method = 'update',
#                  args = [{'visible': UAS_dsm_b},
#                          {'title': 'UAS DEM'}]),
              ]),
    )
    ])

In [14]:
layout = go.Layout(
    title='Cotton Brook Landslide',
    autosize=True,
    width=1800,
    height=1000,
    updatemenus=updatemenus,
    scene = dict(
           xaxis=dict(),
           yaxis=dict(),
           zaxis=dict(),
           aspectmode='manual', # this can be 'data', 'cube', 'auto', 'manual'
           # custom aspect ratio
           aspectratio=dict(x=1, y=1, z=0.3)
           )
    )

In [15]:
# Create and Render Figure

plot_figure = go.Figure(data=[surf, surf2, extent_trace], layout=layout)

# Render the plot.
init_notebook_mode(connected=True)
#plotly.offline.iplot(plot_figure)
plotly.offline.plot(plot_figure,)

'temp-plot.html'